In [124]:
import numpy as np 
import pandas as pd 
HORIZON = 28 # horizon de forecast
CAT_COLS = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
np.random.seed(25)

row_sales = pd.read_csv("C:/Users/n000193384/Downloads/awa_mooc/src/data/sales_train_evaluation.csv")
sample_ts = np.random.choice(row_sales['id'], (750,))

skip = [f"d_{i}" for i in range(1, 700 + 1)] 
row_sales = row_sales.loc[row_sales['id'].isin(sample_ts)].drop(skip, axis=1)


calendar = pd.read_csv("C:/Users/n000193384/Downloads/awa_mooc/src/data/calendar.csv", 
                       parse_dates=['date'],
                       usecols=['date', 'd', 'wm_yr_wk', 'event_name_1', 'event_name_2', 
                                'event_type_1', 'event_type_2',
                                'snap_CA', 'snap_TX', 'snap_WI']
)

# formattage ligne et renommage des colonnes pour respecter les conventions de la librairie.
train = (row_sales
         .pipe(pd.melt, 
               id_vars=["id"] + CAT_COLS,
               var_name='d',
               value_name="y"
               )
         .merge(calendar, how="left", on=["d"])
         .assign(date= lambda x : pd.to_datetime(x['date']))
        )

train.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_701,5,2012-12-29,11249,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_033_CA_1_evaluation,HOBBIES_1_033,HOBBIES_1,HOBBIES,CA_1,CA,d_701,0,2012-12-29,11249,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_150_CA_1_evaluation,HOBBIES_1_150,HOBBIES_1,HOBBIES,CA_1,CA,d_701,5,2012-12-29,11249,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_165_CA_1_evaluation,HOBBIES_1_165,HOBBIES_1,HOBBIES,CA_1,CA,d_701,0,2012-12-29,11249,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_336_CA_1_evaluation,HOBBIES_1_336,HOBBIES_1,HOBBIES,CA_1,CA,d_701,0,2012-12-29,11249,NaN,NaN,NaN,NaN,0,0,0


# Let's forecast the next 28 days with minimal features and explore different encoding method

In [125]:
end_validation = train["date"].max()
start_validation = end_validation - pd.DateOffset(days=HORIZON)

We will define direct forecasting model, 28 lags and encoding feature + few temporal feature such as [dow, week, month]

In [126]:
lags_feat = [f"lag{i+HORIZON}" for i in np.arange(HORIZON+1)]
temporal_feat = ["month", "dow", "week"] 

# lags.
X = (train
     .sort_values(by=["id", "date"])
     .assign(
         month = lambda x : x["date"].dt.month,
         dow = lambda x : x["date"].dt.dayofweek,
         week = lambda x : np.int64(x["date"].dt.isocalendar().week),
         **{f"lag{i+HORIZON}":train.groupby('id')["y"].transform(lambda x : x.shift(i+HORIZON)) for i in np.arange(HORIZON+1)})
     )[["id", "date", "y"] +  CAT_COLS + temporal_feat + lags_feat].copy()

X_tr = X.loc[X["date"] < start_validation]
X_tst = X.loc[X["date"] >= start_validation]

LabelEncoding 

In [127]:
import lightgbm as lgb 
from sklearn.metrics import mean_absolute_error

model_params = {
    'verbose': -1,
    'force_col_wise': True,
    'num_leaves': 100,
    'n_estimators': 250,
}
model = lgb.LGBMRegressor(**model_params)

In [128]:
for c in CAT_COLS:
    print(f"{c} = {X[c].nunique()}")

item_id = 663
dept_id = 7
cat_id = 3
store_id = 10
state_id = 3


In [129]:
cardinal_cat_cols = ['item_id', 'store_id']

# Label Encoding

In [130]:
from sklearn.preprocessing import LabelEncoder

Lb = LabelEncoder()
lb_feat = []
for c in cardinal_cat_cols:
    X_tr[f'lb_{c}'] = Lb.fit_transform(X_tr[c])
    X_tst[f"lb_{c}"] = Lb.transform(X_tst[c])
    lb_feat.append(f"lb_{c}")
    
model.fit(X_tr[temporal_feat + lags_feat + lb_feat].astype(float), X_tr['y'])
yhat = model.predict(X_tst[temporal_feat + lags_feat + lb_feat].astype(float))
mae = mean_absolute_error(X_tst['y'], yhat, sample_weight=X_tst["y"])
bias = np.mean(yhat - X_tst['y'])

print(f"mean absolute erro is {mae} and bias equals to {bias}")

C:\Users\n000193384\AppData\Local\Temp\ipykernel_2744\1383321429.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tr[f'lb_{c}'] = Lb.fit_transform(X_tr[c])
C:\Users\n000193384\AppData\Local\Temp\ipykernel_2744\1383321429.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tst[f"lb_{c}"] = Lb.transform(X_tst[c])
C:\Users\n000193384\AppData\Local\Temp\ipykernel_2744\1383321429.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

mean absolute erro is 5.476048650982964 and bias equals to -0.21834726003399485


# Onehot Encoding

In [131]:
cp_X = X.copy()
cols_dict = {}
for c in cardinal_cat_cols:
    ohe_ds = pd.get_dummies(X[c], prefix="ohe").astype(np.uint8)
    ohe_cols = ohe_ds.columns.tolist()
    cp_X = pd.concat((cp_X, ohe_ds), axis=1)
    cols_dict[c] = ohe_cols
    del ohe_ds
    

X_tr = cp_X.loc[cp_X["date"] < start_validation]
X_tst = cp_X.loc[cp_X["date"] >= start_validation]

ohe_feat = cols_dict['item_id'] + cols_dict['store_id']

model.fit(X_tr[temporal_feat + lags_feat + ohe_feat], X_tr['y'])
yhat = model.predict(X_tst[temporal_feat + lags_feat + ohe_feat])
mae = mean_absolute_error(X_tst['y'], yhat, sample_weight=X_tst["y"])
bias = np.mean(yhat - X_tst['y'])

print(f"mean absolute erro is {mae} and bias equals to {bias}")

mean absolute erro is 5.410876897774181 and bias equals to -0.19952225117447575


# STATIC Target Encoding 

In [132]:
ste_feat = []
for c in cardinal_cat_cols:
    fname = f"te_{c}_mean"
    te_feat = X_tr.groupby(c)["y"].mean().to_frame(fname).reset_index()
    X_tr = pd.merge(X_tr, te_feat, how="left", on=[c])
    X_tst = pd.merge(X_tst, te_feat, how="left", on=[c])
    ste_feat.append(fname)
    

model.fit(X_tr[temporal_feat + lags_feat + ste_feat].astype(float), X_tr['y'])
yhat = model.predict(X_tst[temporal_feat + lags_feat + ste_feat].astype(float))
mae = mean_absolute_error(X_tst['y'], yhat, sample_weight=X_tst["y"])
bias = np.mean(yhat - X_tst['y'])

print(f"mean absolute erro is {mae} and bias equals to {bias}")

mean absolute erro is 5.288595300345391 and bias equals to -0.24375465440947097


# Sliced target encoding 

In [133]:
ste_feat = []
for c in cardinal_cat_cols:
    fname = f"te_{c}_last_28d_mean"
    te_feat = X_tr.groupby(c).nth(slice(-28, None)).groupby(c)["y"].mean().to_frame(fname).reset_index()
    X_tr = pd.merge(X_tr, te_feat, how="left", on=c)
    X_tst = pd.merge(X_tst, te_feat, how="left", on=c)
    ste_feat.append(fname)
    

model.fit(X_tr[temporal_feat + lags_feat + ste_feat].astype(float), X_tr['y'])
yhat = model.predict(X_tst[temporal_feat + lags_feat + ste_feat].astype(float))
mae = mean_absolute_error(X_tst['y'], yhat, sample_weight=X_tst["y"])
bias = np.mean(yhat - X_tst['y'])

print(f"mean absolute erro is {mae} and bias equals to {bias}")

mean absolute erro is 5.542734322347635 and bias equals to -0.22450494464600132


# Temporal Target Encoding

In [134]:
wte_feat = []
for c in cardinal_cat_cols:
    for w in [28, 56, 90, 180]:
        X[f"windowed_te_{w}_{c}"] = X.groupby(c)['y'].transform(lambda x : x.shift(HORIZON).rolling(w).mean())
        wte_feat.append(f"windowed_te_{w}_{c}")
    
X_tr = X.loc[X["date"] < start_validation]
X_tst = X.loc[X["date"] >= start_validation]

model.fit(X_tr[temporal_feat + lags_feat + wte_feat].astype(float), X_tr['y'])
yhat = model.predict(X_tst[temporal_feat + lags_feat + wte_feat].astype(float))
mae = mean_absolute_error(X_tst['y'], yhat, sample_weight=X_tst["y"])
bias = np.mean(yhat - X_tst['y'])

print(f"mean absolute error is {mae} and bias equals to {bias}")

mean absolute error is 5.185346477935068 and bias equals to -0.1781395220580267


# Trend Encoding

In [138]:
from sklearn.linear_model import LinearRegression
from joblib import Parallel, delayed

def simple_lr_trend(df, uid):
    X = np.array((df["date"] - df["date"].min()).dt.days)[:, None]
    y = np.log1p(df["y"])
    model = LinearRegression()
    model.fit(X, y)
    trend = model.predict(X)
    futures_dates = pd.date_range(df["date"].max()+ pd.DateOffset(days=1), periods=28+1, freq="D")
    X_test = np.array((futures_dates - df["date"].min()).days)[:, None]
    test_trend_projec = model.predict(X_test)
    shell = pd.DataFrame()
    shell["date"] = pd.date_range(df["date"].min(), futures_dates[-1], freq="D")
    shell["uid"] =  uid
    shell["trend"] = np.concatenate((trend, test_trend_projec), axis=0)
    return shell
     

In [139]:
for c in cardinal_cat_cols:
    c_df = X_tr.groupby(["date"] + [c])['y'].sum().reset_index()
    uid = c_df[c].unique()
    with Parallel(n_jobs=-1) as parralel:
        all_feats = parralel(delayed(simple_lr_trend)(c_df.query(f"{c} == @ts_id"), ts_id) for ts_id in uid)
    all_feats_conc = pd.concat(all_feats)
    all_feats_conc.columns = ["date", c, f"{c}_trend"]
    X_tr = X_tr.merge(all_feats_conc, how="left", on=["date", c])
    X_tst = X_tst.merge(all_feats_conc, how="left", on=["date", c])
    

In [141]:
trend_feat = ["item_id_trend_y", "store_id_trend_y"] 

model.fit(X_tr[temporal_feat + lags_feat + trend_feat].astype(float), X_tr['y'])
yhat = model.predict(X_tst[temporal_feat + lags_feat + trend_feat].astype(float))
mae = mean_absolute_error(X_tst['y'], yhat, sample_weight=X_tst["y"])
bias = np.mean(yhat - X_tst['y'])

print(f"mean absolute error is {mae} and bias equals to {bias}")

mean absolute error is 5.284021953529792 and bias equals to -0.16944617796217082
